In [1]:
"""
Intuitionistic Natural Deduction Proof Assistant
Single-file Python implementation.

Features:
- Formula AST (Atomic, Implication, Conjunction, Disjunction, Negation, Top, Bottom)
- Recursive-descent parser for formula strings (->, /\\, \/, ~, T, F)
- ProofNode tree structure with inference rules: Axiom (assumption), ImpI, ImpE, AndI, AndE1/AndE2, OrI1/OrI2, OrE, BotE
- Checking/validation of derivations in intuitionistic natural deduction
- Normalization (cut-elimination) to remove detours: beta-reduction like transformations
- Pretty printing of proofs and proofs with discharged assumptions
- Examples at the bottom

This is designed for pedagogy and moderate experimentation, not for full formalization like Lean.
"""
from __future__ import annotations
from dataclasses import dataclass, field
from typing import List, Optional, Tuple, Set, Dict, Union
import re
import copy

# ---------- Formula AST ----------
class Formula:
    def __eq__(self, other):
        return isinstance(other, Formula) and repr(self)==repr(other)

    def __hash__(self):
        return hash(repr(self))

@dataclass(frozen=True)
class Atom(Formula):
    name: str
    def __repr__(self):
        return self.name

@dataclass(frozen=True)
class Imp(Formula):
    left: Formula
    right: Formula
    def __repr__(self):
        return f"({self.left} -> {self.right})"

@dataclass(frozen=True)
class And(Formula):
    left: Formula
    right: Formula
    def __repr__(self):
        return f"({self.left} /\\ {self.right})"

@dataclass(frozen=True)
class Or(Formula):
    left: Formula
    right: Formula
    def __repr__(self):
        return f"({self.left} \/ {self.right})"

@dataclass(frozen=True)
class Not(Formula):
    f: Formula
    def __repr__(self):
        return f"(~{self.f})"

@dataclass(frozen=True)
class Bot(Formula):
    def __repr__(self):
        return "F"

@dataclass(frozen=True)
class Top(Formula):
    def __repr__(self):
        return "T"

# ---------- Parser (recursive descent) ----------
# Grammar (informal):
#  formula := implication
#  implication := disjunction ( '->' implication )?
#  disjunction := conjunction ( '\/' conjunction )*
#  conjunction := atom ( '/\\' atom )*
#  atom := '(' formula ')' | '~' atom | IDENT | 'T' | 'F'

class ParserError(Exception): pass

class Parser:
    def __init__(self, s: str):
        self.s = s
        self.i = 0
        self.token_re = re.compile(r"\s*(->|\\/|/\\|\\\(|\\\)|~|T|F|[A-Za-z][A-Za-z0-9_]*)")
        self.tokens = self._tokenize()
        self.pos = 0

    def _tokenize(self):
        tokens = []
        i = 0
        while i < len(self.s):
            m = self.token_re.match(self.s, i)
            if not m:
                raise ParserError(f"Cannot tokenize at: {self.s[i:]} (pos {i})")
            tok = m.group(1)
            tokens.append(tok)
            i = m.end()
        tokens.append(None)
        return tokens

    def peek(self):
        return self.tokens[self.pos]

    def consume(self, tok=None):
        cur = self.peek()
        if tok is not None and cur != tok:
            raise ParserError(f"Expected {tok!r} but found {cur!r}")
        self.pos += 1
        return cur

    def parse(self) -> Formula:
        f = self.parse_implication()
        if self.peek() is not None:
            raise ParserError(f"Extra token {self.peek()}")
        return f

    def parse_implication(self) -> Formula:
        left = self.parse_disjunction()
        if self.peek() == '->':
            self.consume('->')
            right = self.parse_implication()
            return Imp(left, right)
        return left

    def parse_disjunction(self) -> Formula:
        f = self.parse_conjunction()
        while self.peek() == '\\/':
            self.consume('\\/')
            g = self.parse_conjunction()
            f = Or(f, g)
        return f

    def parse_conjunction(self) -> Formula:
        f = self.parse_atom()
        while self.peek() == '/\\':
            self.consume('/\\')
            g = self.parse_atom()
            f = And(f, g)
        return f

    def parse_atom(self) -> Formula:
        p = self.peek()
        if p == '(':
            self.consume('(')
            f = self.parse_implication()
            self.consume(')')
            return f
        if p == '~':
            self.consume('~')
            inner = self.parse_atom()
            return Not(inner)
        if p == 'T':
            self.consume('T')
            return Top()
        if p == 'F':
            self.consume('F')
            return Bot()
        if isinstance(p, str) and re.match(r'[A-Za-z]', p):
            self.consume()
            return Atom(p)
        raise ParserError(f"Unexpected token {p}")

# convenience

def parse_formula(s: str) -> Formula:
    return Parser(s).parse()

# ---------- Proof nodes and rules ----------

@dataclass
class ProofNode:
    rule: str
    conclusion: Formula
    premises: List['ProofNode'] = field(default_factory=list)
    discharged: List[str] = field(default_factory=list)  # labels of assumptions discharged by this node
    label: Optional[str] = None  # label for assumptions (if rule is 'assume')
    meta: dict = field(default_factory=dict)

    def copy(self):
        return copy.deepcopy(self)

    def pretty(self, indent=0) -> str:
        pad = '  ' * indent
        lab = f" [{self.label}]" if self.label else ''
        disc = f" (discharged: {self.discharged})" if self.discharged else ''
        s = f"{pad}{self.rule}{lab}: {self.conclusion}{disc}\n"
        for p in self.premises:
            s += p.pretty(indent+1)
        return s

# Builder helpers

def assume(formula: Formula, label: Optional[str]=None) -> ProofNode:
    return ProofNode(rule='Assume', conclusion=formula, premises=[], label=label)

def imp_i(subproof: ProofNode, assumption_label: str) -> ProofNode:
    # discharges assumption_label and builds Imp(A,B)
    A = None
    # find the assumption formula with that label in the subproof assumption set
    A = find_assumption_formula_in(subproof, assumption_label)
    if A is None:
        raise ValueError(f"Unable to find assumption label {assumption_label} in subproof")
    node = ProofNode(rule='->I', conclusion=Imp(A, subproof.conclusion), premises=[subproof], discharged=[assumption_label])
    return node

def imp_e(pf_imp: ProofNode, pf_arg: ProofNode) -> ProofNode:
    return ProofNode(rule='->E', conclusion=pf_imp.conclusion.right if isinstance(pf_imp.conclusion, Imp) else None, premises=[pf_imp, pf_arg])

def and_i(pf1: ProofNode, pf2: ProofNode) -> ProofNode:
    return ProofNode(rule='/\\I', conclusion=And(pf1.conclusion, pf2.conclusion), premises=[pf1, pf2])

def and_e1(pf: ProofNode) -> ProofNode:
    if not isinstance(pf.conclusion, And):
        return ProofNode(rule='/\\E1', conclusion=None, premises=[pf])
    return ProofNode(rule='/\\E1', conclusion=pf.conclusion.left, premises=[pf])

def and_e2(pf: ProofNode) -> ProofNode:
    if not isinstance(pf.conclusion, And):
        return ProofNode(rule='/\\E2', conclusion=None, premises=[pf])
    return ProofNode(rule='/\\E2', conclusion=pf.conclusion.right, premises=[pf])

def or_i1(pf: ProofNode, right_formula: Formula) -> ProofNode:
    return ProofNode(rule='\\/I1', conclusion=Or(pf.conclusion, right_formula), premises=[pf])

def or_i2(left_formula: Formula, pf: ProofNode) -> ProofNode:
    return ProofNode(rule='\\/I2', conclusion=Or(left_formula, pf.conclusion), premises=[pf])

def or_e(pf_or: ProofNode, sub1: ProofNode, sub2: ProofNode) -> ProofNode:
    # pf_or: p \/ q; sub1: assume p [a] ... conclude r; sub2: assume q [b] ... conclude r
    if not isinstance(pf_or.conclusion, Or):
        return ProofNode(rule='\\/E', conclusion=None, premises=[pf_or, sub1, sub2])
    return ProofNode(rule='\\/E', conclusion=sub1.conclusion, premises=[pf_or, sub1, sub2], discharged=[sub1.label, sub2.label])

def bot_e(pf: ProofNode, target: Formula) -> ProofNode:
    return ProofNode(rule='F_E', conclusion=target, premises=[pf])

# utilities to find assumption with label

def find_assumption_formula_in(pf: ProofNode, label: str) -> Optional[Formula]:
    # DFS to find an assumption node with that label
    stack = [pf]
    seen = set()
    while stack:
        node = stack.pop()
        if id(node) in seen: continue
        seen.add(id(node))
        if node.rule == 'Assume' and node.label == label:
            return node.conclusion
        stack.extend(node.premises)
    return None

# ---------- Type checking / validation ----------

class CheckError(Exception): pass

def check(pf: ProofNode, context: Optional[Dict[str, Formula]]=None) -> Tuple[Formula, Set[str]]:
    """
    Recursively checks the proof node. Returns (conclusion_formula, set of open assumption labels)
    Raises CheckError on failure.
    """
    if context is None:
        context = {}
    rule = pf.rule
    if rule == 'Assume':
        # assumption introduces an open assumption with label
        return pf.conclusion, set([pf.label]) if pf.label else set()
    elif rule == '->I':
        # premise is a subproof; it must discharge an assumption label present
        sub = pf.premises[0]
        f_sub, open_labels = check(sub, context)
        # pf.discharged should contain exactly one label
        if not pf.discharged or len(pf.discharged) != 1:
            raise CheckError("->I must discharge exactly one label")
        lab = pf.discharged[0]
        if lab not in open_labels:
            raise CheckError(f"->I discharges label {lab} not present in subproof")
        # result is implication from formula_of_lab to sub.conclusion
        A = find_assumption_formula_in(sub, lab)
        if A is None:
            raise CheckError(f"Cannot locate formula for discharged label {lab}")
        # remaining open labels are those except lab
        open_labels.remove(lab)
        # ensure pf.conclusion matches
        expected = Imp(A, f_sub)
        if pf.conclusion != expected:
            raise CheckError(f"->I conclusion expected {expected} found {pf.conclusion}")
        return pf.conclusion, open_labels
    elif rule == '->E':
        imp, arg = pf.premises
        f_imp, open1 = check(imp, context)
        f_arg, open2 = check(arg, context)
        # imp must be implication
        if not isinstance(f_imp, Imp):
            raise CheckError("->E left premise not an implication")
        if f_imp.left != f_arg:
            raise CheckError(f"->E mismatch: function expects {f_imp.left} got {f_arg}")
        if pf.conclusion != f_imp.right:
            raise CheckError(f"->E conclusion mismatch: expected {f_imp.right} got {pf.conclusion}")
        return pf.conclusion, open1.union(open2)
    elif rule == '/\\I':
        p1, p2 = pf.premises
        f1, o1 = check(p1, context)
        f2, o2 = check(p2, context)
        if pf.conclusion != And(f1, f2):
            raise CheckError(f"/\\I conclusion mismatch: expected {(f1,f2)} got {pf.conclusion}")
        return pf.conclusion, o1.union(o2)
    elif rule == '/\\E1':
        p1 = pf.premises[0]
        f1, o1 = check(p1, context)
        if not isinstance(f1, And):
            raise CheckError('/\\E1 expects a conjunction premise')
        if pf.conclusion != f1.left:
            raise CheckError(f"/\\E1 conclusion mismatch: expected {f1.left} got {pf.conclusion}")
        return pf.conclusion, o1
    elif rule == '/\\E2':
        p1 = pf.premises[0]
        f1, o1 = check(p1, context)
        if not isinstance(f1, And):
            raise CheckError('/\\E2 expects a conjunction premise')
        if pf.conclusion != f1.right:
            raise CheckError(f"/\\E2 conclusion mismatch: expected {f1.right} got {pf.conclusion}")
        return pf.conclusion, o1
    elif rule in ('\\/I1','\\/I2'):
        p1 = pf.premises[0]
        f1, o1 = check(p1, context)
        if pf.rule == '\\/I1' and not isinstance(pf.conclusion, Or):
            raise CheckError('\\/I1 conclusion must be disjunction')
        if pf.rule == '\\/I2' and not isinstance(pf.conclusion, Or):
            raise CheckError('\\/I2 conclusion must be disjunction')
        # we don't strictly check right/left membership here
        return pf.conclusion, o1
    elif rule == '\\/E':
        por = pf.premises[0]
        s1 = pf.premises[1]
        s2 = pf.premises[2]
        f_or, o_or = check(por, context)
        f1, o1 = check(s1, context)
        f2, o2 = check(s2, context)
        if not isinstance(f_or, Or):
            raise CheckError('\\/E expects a disjunction as first premise')
        # s1 and s2 should be subproofs whose assumptions are labeled as s1.label and s2.label respectively
        if not s1.label or not s2.label:
            raise CheckError('Subproofs in \/E must be assumptions with labels')
        # ensure they conclude the same formula
        if f1 != f2:
            raise CheckError('Branches in \/E must conclude the same formula')
        if pf.conclusion != f1:
            raise CheckError('\\/E conclusion must match branch conclusions')
        # ensure discharged labels are present
        if not pf.discharged or len(pf.discharged) != 2:
            raise CheckError('\\/E must list two discharged labels')
        # ensure that s1.label and s2.label present in their open sets
        if s1.label not in o1 or s2.label not in o2:
            raise CheckError('\\/E branch labels not present in branch open assumptions')
        # union of open assumptions: all except the two branch assumptions which are discharged
        opens = o_or.union(o1).union(o2)
        opens.discard(s1.label); opens.discard(s2.label)
        return pf.conclusion, opens
    elif rule == 'F_E':
        p1 = pf.premises[0]
        f1, o1 = check(p1, context)
        if not isinstance(f1, Bot):
            raise CheckError('F_E requires bottom as premise')
        return pf.conclusion, o1
    else:
        raise CheckError(f"Unknown rule {rule}")

# ---------- Normalization / Cut-elimination (simplified) ----------

def normalize(pf: ProofNode) -> ProofNode:
    """
    Perform local detour eliminations:
    - If node is ->E where left is ->I that discharged label a, then inline (beta-reduction)
    - If node is /\\E1 or /\\E2 applied to /\\I, reduce
    This is a recursive one-pass normalization; fixed point applied until stable.
    """
    changed = True
    current = pf.copy()
    while True:
        new, did = _normalize_once(current)
        current = new
        if not did:
            break
    return current

def _normalize_once(pf: ProofNode) -> Tuple[ProofNode,bool]:
    # recursively try to normalize children first
    changed = False
    for i, ch in enumerate(pf.premises):
        newch, did = _normalize_once(ch)
        if did:
            pf.premises[i] = newch
            changed = True
    # now attempt local reductions at pf
    if pf.rule == '->E':
        left = pf.premises[0]
        right = pf.premises[1]
        if left.rule == '->I' and left.discharged:
            # beta reduction: (\a. M) N  -->  M[N/a]
            lab = left.discharged[0]
            # substitute occurrences of assumption labeled lab in left.premises[0] with right
            body = left.premises[0]
            # perform substitution of assumption nodes with label lab by right
            new_body = substitute_assumption(body, lab, right)
            # the result is new_body (but we must preserve any remaining discharged labels)
            return new_body.copy(), True
    if pf.rule in ('/\\E1','/\\E2'):
        ch = pf.premises[0]
        if ch.rule == '/\\I':
            # projection of a constructed conjunction
            if pf.rule == '/\\E1':
                return ch.premises[0].copy(), True
            else:
                return ch.premises[1].copy(), True
    return pf, changed

def substitute_assumption(node: ProofNode, label: str, replacement: ProofNode) -> ProofNode:
    """
    Replace assumption nodes labeled `label` in `node` by `replacement` (copying replacement each time).
    Also remove discharged labels accordingly.
    """
    node = node.copy()
    def rec(n: ProofNode) -> ProofNode:
        if n.rule == 'Assume' and n.label == label:
            # replace with copy of replacement
            return replacement.copy()
        # otherwise recurse
        n.premises = [rec(ch) for ch in n.premises]
        # remove discharged labels from nodes if present
        n.discharged = [d for d in n.discharged if d != label]
        return n
    return rec(node)

# ---------- Human-readable exporter ----------

def pretty_print_proof(pf: ProofNode) -> str:
    try:
        check(pf)
    except Exception as e:
        pre = "(unchecked or invalid proof)\n"
        return pre + pf.pretty()
    return pf.pretty()

# ---------- Examples ----------

if __name__ == '__main__':
    # Simple proofs as demonstrations
    A = parse_formula('A')
    B = parse_formula('B')

    # Proof of A -> A
    a1 = assume(A, label='a')
    imp = imp_i(a1, 'a')
    print('Proof of A -> A:')
    print(pretty_print_proof(imp))

    # Modus ponens: from A -> B and A infer B
    pa = assume(A, label='p')
    imp_ab = assume(Imp(A,B), label='q')
    mp = imp_e(imp_ab, pa)
    print('\nModus ponens (unchecked until explicit check):')
    print(pretty_print_proof(mp))

    # Conjunction introduction and elimination
    a2 = assume(A, label='x')
    b2 = assume(B, label='y')
    conj = and_i(a2, b2)
    proj = and_e1(conj)
    print('\nConjunction example:')
    print(pretty_print_proof(conj))
    print(pretty_print_proof(proj))

    # Normalization demo: (\a. a) applied to A reduces to A
    # That is, imp_i(a1,'a') applied to pa
    red = imp_e(imp, pa)
    print('\nBefore normalization:')
    print(pretty_print_proof(red))
    norm = normalize(red)
    print('\nAfter normalization:')
    print(pretty_print_proof(norm))

    # Disjunction: from A infer A \/ B
    or1 = or_i1(a2, B)
    print('\nDisjunction intro:')
    print(pretty_print_proof(or1))

    # Note: full \/E (case) requires building branches that assume each disjunct and lead to same conclusion.
    # Example: from (A \/ B), and (A -> C), (B -> C) infer C by \/E.
    c = parse_formula('C')
    imp_ac = assume(Imp(A,c), label='r')
    imp_bc = assume(Imp(B,c), label='s')
    # build branches: assume A [u] ... use imp_ac to get C; similarly for B
    br1_ass = assume(A, label='u')
    br1 = imp_e(imp_ac, br1_ass)
    br2_ass = assume(B, label='v')
    br2 = imp_e(imp_bc, br2_ass)
    por = assume(Or(A,B), label='o')
    case = or_e(por, br1_ass, br2_ass)  # build or elimination node (note: simplified)
    print('\nDisjunction elimination (simplified node):')
    print(pretty_print_proof(case))

    # End of demonstration
    print('\nEnd of demo. This assistant supports building, checking, and normalizing proofs in an intuitionistic ND style.')


Proof of A -> A:
->I: (A -> A) (discharged: ['a'])
  Assume [a]: A


Modus ponens (unchecked until explicit check):
->E: B
  Assume [q]: (A -> B)
  Assume [p]: A


Conjunction example:
/\I: (A /\ B)
  Assume [x]: A
  Assume [y]: B

/\E1: A
  /\I: (A /\ B)
    Assume [x]: A
    Assume [y]: B


Before normalization:
->E: A
  ->I: (A -> A) (discharged: ['a'])
    Assume [a]: A
  Assume [p]: A


After normalization:
Assume [p]: A


Disjunction intro:
\/I1: (A \/ B)
  Assume [x]: A


Disjunction elimination (simplified node):
(unchecked or invalid proof)
\/E: A (discharged: ['u', 'v'])
  Assume [o]: (A \/ B)
  Assume [u]: A
  Assume [v]: B


End of demo. This assistant supports building, checking, and normalizing proofs in an intuitionistic ND style.


<>:7: SyntaxWarning: invalid escape sequence '\/'
<>:55: SyntaxWarning: invalid escape sequence '\/'
<>:346: SyntaxWarning: invalid escape sequence '\/'
<>:349: SyntaxWarning: invalid escape sequence '\/'
<>:7: SyntaxWarning: invalid escape sequence '\/'
<>:55: SyntaxWarning: invalid escape sequence '\/'
<>:346: SyntaxWarning: invalid escape sequence '\/'
<>:349: SyntaxWarning: invalid escape sequence '\/'
/tmp/ipython-input-2159449699.py:7: SyntaxWarning: invalid escape sequence '\/'
  - Recursive-descent parser for formula strings (->, /\\, \/, ~, T, F)
/tmp/ipython-input-2159449699.py:55: SyntaxWarning: invalid escape sequence '\/'
  return f"({self.left} \/ {self.right})"
/tmp/ipython-input-2159449699.py:346: SyntaxWarning: invalid escape sequence '\/'
  raise CheckError('Subproofs in \/E must be assumptions with labels')
/tmp/ipython-input-2159449699.py:349: SyntaxWarning: invalid escape sequence '\/'
  raise CheckError('Branches in \/E must conclude the same formula')
